In [ ]:
!pip install langgraph

In [5]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List

class ParallelEvaluationState(TypedDict):
    resume_text: str
    technical_score: int
    cultural_score: int
    experience_score: int
    all_evaluations: List[dict]
    final_decision: str

def evaluate_technical_skills(state: ParallelEvaluationState) -> dict:
    print("Evaluating technical skills...")
    resume = state["resume_text"]
    tech_keywords = ["python", "java", "sql", "aws"]
    score = sum(20 for keyword in tech_keywords if keyword in resume.lower())
    return {"technical_score": score}

def evaluate_cultural_fit(state: ParallelEvaluationState) -> dict:
    print("Evaluating cultural fit...")
    resume = state["resume_text"]
    culture_keywords = ["team", "collaboration", "leadership", "agile"]
    score = sum(20 for keyword in culture_keywords if keyword in resume.lower())
    return {"cultural_score": score}

def evaluate_experience(state: ParallelEvaluationState) -> dict:
    print("Evaluating experience...")
    resume = state["resume_text"]
    if "senior" in resume.lower() or "10 years" in resume.lower():
        score = 100
    elif "5 years" in resume.lower():
        score = 70
    else:
        score = 40
    return {"experience_score": score}

def aggregate_results(state: ParallelEvaluationState) -> dict:
    print("Aggregating all evaluations...")
    evaluations = [
        {"aspect": "Technical", "score": state["technical_score"]},
        {"aspect": "Cultural", "score": state["cultural_score"]},
        {"aspect": "Experience", "score": state["experience_score"]}
    ]
    avg_score = sum(e["score"] for e in evaluations) / len(evaluations)

    if avg_score >= 70:
        decision = "Strong Candidate - Schedule Interview"
    elif avg_score >= 50:
        decision = "Potential Candidate - Additional Review"
    else:
        decision = "Not a Match"

    return {"all_evaluations": evaluations, "final_decision": decision}

workflow = StateGraph(ParallelEvaluationState)
workflow.add_node("eval_tech", evaluate_technical_skills)
workflow.add_node("eval_culture", evaluate_cultural_fit)
workflow.add_node("eval_experience", evaluate_experience)
workflow.add_node("aggregate", aggregate_results)


workflow.set_entry_point("parallel_entry")
workflow.add_node("parallel_entry", lambda s: s)

# Connect parallel entry to evaluation nodes
workflow.add_edge("parallel_entry", "eval_tech")
workflow.add_edge("parallel_entry", "eval_culture")
workflow.add_edge("parallel_entry", "eval_experience")

# Connect evaluation nodes to the aggregate node - this implicitly handles the join
workflow.add_edge("eval_tech", "aggregate")
workflow.add_edge("eval_culture", "aggregate")
workflow.add_edge("eval_experience", "aggregate")


workflow.add_edge("aggregate", END)

app = workflow.compile()

test_resume = "Senior Python developer with 10 years experience, strong team player, agile practitioner, AWS certified."
initial_state = {"resume_text": test_resume}
result = app.invoke(initial_state)

print(f"\nFinal Decision: {result['final_decision']}")
print("Scores breakdown:")
for eval in result['all_evaluations']:
    print(f"  {eval['aspect']}: {eval['score']}")

Evaluating cultural fit...
Evaluating experience...
Evaluating technical skills...
Aggregating all evaluations...

Final Decision: Potential Candidate - Additional Review
Scores breakdown:
  Technical: 40
  Cultural: 40
  Experience: 100
